## Usage examples for `py.ciu.image` package

This code block contains all the `pip install`s that were executed when starting with a (more or less) empty environment. Included mainly FYI. 

In [ ]:
# Take care of all requirements. Comment away these if you have already installed as it should already!
#!python3 -m pip install --upgrade pip
#!pip install pandas
#!pip install opencv-python
#!pip install matplotlib
#!pip install tensorflow
#!pip install scikit-image

# These are only needed if you want to develop source files directly in Jupyter Notebook
#!pip install ipython --upgrade 
#%load_ext autoreload 

Do all necessary imports

In [ ]:
#%autoreload 2
import numpy as np
import time
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

from CIU import CIU, SlicOcclusionPerturber, SuperpixelPerturber, SlicSegmenter, EntireSegmentStrategy, GaussianBlurDistortion, SingleColorDistortion, ReplaceImageDistortion, TransformDistortion
from skimage.filters import gaussian, hessian
from skimage.util import random_noise

In [ ]:

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from tensorflow import keras
from tensorflow.keras.utils import custom_object_scope, load_img, img_to_array
from keras.initializers import glorot_uniform
from keras.preprocessing import image
from keras.applications import inception_v3 as inc_net

### Detecting bleeding in gastro-enterological images

Short example that takes pre-trained model, runs CIU on an image instance and then plots the original image and the image where only superpixels with bleeding detected are shown.
We start by loading model and image to explain, as well as the function for manipulating all images to the format, colors etc expected by the model. 

In [ ]:
# Load model and set all needed parameters
with custom_object_scope({'GlorotUniform': glorot_uniform()}):
   model = keras.models.load_model('model_full_categorical.h5')
#model = tf.keras.models.load_model("model_full_categorical.h5")
out_names = ["NonBleeding", "Bleeding"]

# Images should pass through this function before they are given to the model.
def transform_img_fn(path_list):
    out = []
    for img_path in path_list:
        img = load_img(img_path, target_size=(150, 150))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = inc_net.preprocess_input(x)
        out.append(x)
    return np.vstack(out) / 2 + 0.5

Set up bleeding and non-bleeding images used for testing:

In [ ]:
non_bleeding_imgnames = ["Set1_4.png","Set1_8.png","Set1_22.png","Set1_29.png",
                         "Set1_53.png","Set1_153.png","Set1_157.png","Set1_251.png"]
bleeding_imgnames = ["Set1_674.png","Set1_1128.png","Set1_1151.png","Set1_1201.png",
                     "Set1_1277.png","Set1_2812.jpg","Set1_2879.bmp","Set1_3068.bmp",
                     "Set1_3078.bmp","Set1_3105.bmp","Set1_3141.bmp","Set1_3190.bmp"]
imgnames = non_bleeding_imgnames + bleeding_imgnames
imgpath = "images/medical/"
imgpaths = [imgpath + item for item in imgnames]

# Other approach
imgs_vstack = transform_img_fn(imgpaths)

We display all images to see what they look like:

In [ ]:
# Plot all images
ncol = len(imgpaths)
fig, axes = plt.subplots(nrows=1, ncols=ncol, figsize=(8, 8))
for i in range(0,len(imgs_vstack)): 
    axes[i].imshow(imgs_vstack[i])
    axes[i].axis('off')
    #axes[i].set_title(imgnames[i])
plt.tight_layout() # Adjust spacing between images
plt.show()

However, that becomes too small when the number of images grows bigger, so let's use grid instead:

In [ ]:
import math

# Define the grid layout
num_images_to_display = len(imgpaths)
num_cols = 8
num_rows = math.ceil(num_images_to_display/num_cols)

# Function for plotting images in a grid
def plotgrid(imgs_vstack, nrows, ncols, figsize=(10,4), figtitles=None):
    total_subplots = nrows * ncols
    n_imgs = len(imgs_vstack)
    fig, axs = plt.subplots(nrows, ncols, figsize=figsize)
    for i in range(nrows):
        for j in range(ncols):
            if i * ncols + j < n_imgs:
                axs[i,j].imshow(imgs_vstack[i * ncols + j])
                if figtitles is not None:
                    axs[i,j].set_title(figtitles[i * ncols + j])
                axs[i,j].axis('off')
            else:
                axs[i, j].axis('off')  # Turn off empty subplots
    # Remove any excess empty subplots from the display
    for i in range(n_imgs, total_subplots):
        fig.delaxes(axs.flatten()[i])
    plt.tight_layout() # Adjust spacing between images
    plt.show()
    return

plotgrid(imgs_vstack, num_rows, num_cols, figtitles=imgnames)

Let's see what the model predicts for all the images. We set up a "pred_function" to be used later in order to pass to CIU. CIU is model-agnostic so it needs to know how to get the predicted output values from the model.

In [ ]:
pred_function = model.predict_on_batch
outvals = pred_function(imgs_vstack)
for ov in outvals: 
    print(f"The model output indicates {out_names[np.argmax(ov)]}")

If the prediction function is not "predict_on_batch", it can be given as a parameter to ``CIU()``.

Then we create a CIU object and "explain" why a particular image is "Bleeding" or "NonBleeding". A "NonBleeding" explanation would in this case normally contain all or most of the image, except for the black corners. 

For easier interpretation, we also show the original image.

In [ ]:
img_to_xplain = 13 # Indices start from zero.
image = imgs_vstack[img_to_xplain]
perturber = SlicOcclusionPerturber(strategy="inverse")

# TESTING
segmenter = SlicSegmenter(50, 10)
strategy = EntireSegmentStrategy(inverse=True, fade_fun=gaussian, sigma=2, truncate=2)
color_distortion = SingleColorDistortion((190,190,190))
blur_distortion = GaussianBlurDistortion(sigma=30, truncate=1)
replace_distortion = ReplaceImageDistortion(np.array([imgs_vstack[1]]))#, imgs_vstack[10]]))
hessian_distortion = TransformDistortion(hessian)
noise_distortion = TransformDistortion(random_noise, mode='s&p', amount=0.4) 
#perturber = SuperpixelPerturber(segmenter, strategy, noise_distortion)
# TESTING



ciu_object = CIU(model, out_names, perturber=perturber, inverse=True)
tic = time.perf_counter()
ciu_sp_result = ciu_object.Explain(image)
toc = time.perf_counter()
print(f"Done in {toc - tic:0.4f} seconds")
# Display CI and CU values for "winning" class
winner = np.argmax(ciu_sp_result["outvals"])
print(f"CI values of winner class are:\n {ciu_sp_result['CI'][winner,:]}")
print(f"CU values of winner class are:\n {ciu_sp_result['CU'][winner,:]}")
print(f"Cinfl values of winner class are:\n {ciu_sp_result['Cinfl'][winner,:]}")

It can be useful to display a histogram of CI values for finding a suitable threshold value:

In [ ]:
# Create and show histogram plot
CIU_val = "Cinfl"
vals = ciu_sp_result[CIU_val][winner,:]
plt.hist(vals, bins=30, density=True, alpha=0.6, color='b')
plt.xlabel(f'{CIU_val} value')
plt.ylabel('Frequency')
plt.title(f'Distribution of {CIU_val} values')
plt.show()

Display the corresponding result, together with original image:

In [ ]:
# Include at least one superpixel
max_CI = vals.max()
own_CI_limit = 0.1
img = ciu_object.ImageInfluentialSegmentsOnly(winner, min(max_CI, own_CI_limit))
# Commented out the simple display of "explanation image" only
#plt.imshow(img)
#plt.show()
print(f"The image is {imgnames[img_to_xplain]}")
print(f"The model output indicates {out_names[np.argmax(outvals[img_to_xplain])]}")
fig, axes = plt.subplots(1, 2, figsize=(8,8))
axes[0].imshow(imgs_vstack[img_to_xplain])
axes[0].axis('off')
axes[1].imshow(img)
axes[1].axis('off')
plt.tight_layout() # Adjust spacing between images
plt.show()

Get the segments/superpixels and plot their borders:

In [ ]:
#from scipy import ndimage
from skimage import io, color, segmentation, exposure
from skimage.segmentation import mark_boundaries

# Get the segments
segment_boundaries = segmentation.mark_boundaries(ciu_object.original_image, ciu_object.superpixels)

# Display the original image with superpixel boundaries
plt.figure(figsize=(10, 5))
plt.subplot(121), plt.imshow(ciu_object.original_image), plt.title('Original Image')
plt.subplot(122), plt.imshow(segment_boundaries), plt.title('Superpixel Boundaries')
plt.show()

Get the explanations for all images:

In [ ]:
out_imgs = []
per_imgs = []
seg_imgs = []
def_CI_limit = 0.01
tic = time.perf_counter()
for i in range(0,len(imgpaths)):
    cius = ciu_object.Explain(imgs_vstack[i])
    win_class = np.argmax(cius["outvals"])
    CI_limit = min(def_CI_limit, cius["CI"][win_class,:].max())
    oimg = ciu_object.ImageInfluentialSegmentsOnly(win_class, CI_limit, use_perturber=True)
    pimg = np.squeeze(replace_distortion(np.expand_dims(ciu_object.image, axis=0), ciu_object.get_influential_segments(win_class, CI_limit)), axis=(0,1))
    simg = ciu_object.masks[0][20][0]
    #print(oimg.shape, pimg[0].shape, simg.shape)
    out_imgs.append(oimg)
    per_imgs.append(pimg[0])
    seg_imgs.append(simg)
toc = time.perf_counter()
print(f"Done in {toc - tic:0.4f} seconds")
plotgrid(out_imgs, num_rows, num_cols)
plotgrid(per_imgs, num_rows, num_cols)
plotgrid(seg_imgs, num_rows, num_cols)

Plot original images with names again, for easier comparison. 

In [ ]:
plotgrid(imgs_vstack, num_rows, num_cols, figtitles=imgnames)

The default CI limit wasn't appropriate for images 2812 and 3068. By plotting the histogram of CI values for them (see earlier), we get the following:

In [ ]:
cius1 = ciu_object.Explain(imgs_vstack[13])
win_class = np.argmax(cius1["outvals"])
oimg1 = ciu_object.ImageInfluentialSegmentsOnly(win_class, 0.05)
cius2 = ciu_object.Explain(imgs_vstack[15])
win_class = np.argmax(cius2["outvals"])
oimg2 = ciu_object.ImageInfluentialSegmentsOnly(win_class, 0.05)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(4, 4))
axes[0].imshow(oimg1)
axes[0].axis('off')
axes[1].imshow(oimg2)
axes[1].axis('off')
plt.tight_layout() # Adjust spacing between images
plt.show()